In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# -*- coding: utf-8 -*-
import argparse
import math
import sys
import time
import copy
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization#, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
#K.set_image_dim_ordering('th')
#print(K.image_data_format())

## required for efficient GPU use
#import tensorflow as tf
#from keras.backend import tensorflow_backend
#config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
#session = tf.Session(config=config)
#tensorflow_backend.set_session(session)
## required for efficient GPU use

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
model_path='/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Shallow CNN Model/results/cnn_shallow_train__test__.h5'

# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [4]:
# get data

class dataset:
    kdd_train__2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/KDD Dataset Preprocessing/kdd_train__2labels.pkl") #train-
    kdd_test__2labels = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/KDD Dataset Preprocessing/kdd_test__2labels.pkl") #test-
    
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['is_Normal','is_Attack']
    
    x_input = dataset.kdd_train__2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train__2labels.loc[:,output_columns_2labels]
    
    x_test__input = dataset.kdd_test__2labels.drop(output_columns_2labels, axis = 1)
    y_test_ = dataset.kdd_test__2labels.loc[:,output_columns_2labels]

    ss = pp.StandardScaler()

    x_train_ = ss.fit_transform(x_input)
    x_test_ = ss.transform(x_test__input)

    y_train_ = y_output.values
    y_test_ = y_test_.values
    
    print('x_train shape: {}'.format(x_train_.shape))
    print('y_train shape: {}'.format(y_train_.shape))
    print('x_test_ shape: {}'.format(x_test_.shape))
    print('y_test_ shape: {}'.format(y_test_.shape))

x_train shape: (25192, 122)
y_train shape: (25192, 2)
x_test_ shape: (11850, 122)
y_test_ shape: (11850, 2)


In [5]:
#reshape data
X_train_ = np.reshape(preprocess.x_train_, (preprocess.x_train_.shape[0], preprocess.x_train_.shape[1], 1))
X_test_ = np.reshape(preprocess.x_test_, (preprocess.x_test_.shape[0], preprocess.x_test_.shape[1], 1))

print('train shape after reshape: {}'.format(X_train_.shape))
print('test shape after reshape: {}'.format(X_test_.shape))

train shape after reshape: (25192, 122, 1)
test shape after reshape: (11850, 122, 1)


In [6]:
#size of parameters
batch_size = 32
num_classes = 2
epochs = 4
filter_size= 3
droprate=0.50

In [7]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(122, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))
model.add(MaxPooling1D(strides=2))

#FCN layer
model.add(Flatten())
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           256       
                                                                 
 batch_normalization (BatchN  (None, 122, 64)          256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 122, 64)           0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 61, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3904)              0         
                                                  

In [8]:
#Save Model=ON
history = model.fit(X_train_, preprocess.y_train_,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test_, preprocess.y_test_),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test_, preprocess.y_test_,verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 25192 samples, validate on 11850 samples
Epoch 1/4
25120/25192 [============================>.] - ETA: 0s - loss: 0.1091 - acc: 0.9662

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


25192/25192 [==============================] - 10s 383us/sample - loss: 0.1089 - acc: 0.9662 - val_loss: 1.4663 - val_acc: 0.5520
Epoch 2/4
25192/25192 [==============================] - 9s 359us/sample - loss: 0.0677 - acc: 0.9744 - val_loss: 1.8852 - val_acc: 0.5612
Epoch 3/4
25192/25192 [==============================] - 10s 378us/sample - loss: 0.0580 - acc: 0.9778 - val_loss: 1.8698 - val_acc: 0.5733
Epoch 4/4
25192/25192 [==============================] - 9s 371us/sample - loss: 0.0525 - acc: 0.9798 - val_loss: 1.7661 - val_acc: 0.6139
Test loss: 1.7661424065441018
Test accuracy: 0.613924


In [9]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test_)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

predict_proba = model.predict(X_test_)
predict_classes = np.argmax(predict_proba, axis=1)

target_names = ['Normal', 'Attack']
print(classification_report(np.argmax(preprocess.y_test_, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test_, axis=1), y_pred))

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[9.9977797e-01 2.2202291e-04]
 [9.9674231e-01 3.2577191e-03]
 [9.9988413e-01 1.1591272e-04]
 ...
 [1.4383522e-01 8.5616475e-01]
 [1.7203871e-07 9.9999988e-01]
 [9.9404484e-01 5.9552002e-03]]
[0 0 0 ... 1 1 0]
              precision    recall  f1-score   support

      Normal     0.2690    0.6557    0.3815      2152
      Attack     0.8878    0.6047    0.7194      9698

    accuracy                         0.6139     11850
   macro avg     0.5784    0.6302    0.5504     11850
weighted avg     0.7754    0.6139    0.6580     11850

[[1411  741]
 [3834 5864]]
